In [10]:
import requests
import json
import pandas as pd
import numpy as np
from requests.auth import HTTPBasicAuth
from datetime import datetime
import time
import re
import sys
import numpy as np

owner = "elastic"
repo = "elasticsearch"
# owner = "spring-projects"
# repo = "spring-framework"

delay_conn = 10
clients = [('2cc352a6c740826a4be6', 'daebc120eb033a0e7cd175f72d3b5bee9ebb5b42'),
           ('2aaa427b39670c92148e', 'b1b2204e84d861ab2c306fe317f451ab5e4cc2e4'),
           ('6b324dbf8ccd0c7aee80', '3828eaf02fac452006c4dfa3221a1eaed42c9ab7'),
           ('e42eb9683051f5bde934', '5afda3713ff17f5d99047b67f12aa5ca484abfc3'),
           ('249b7f447a69e0311bae', 'dea9eccf207e3ed899f2074069f04d075ed569b3')]
clients_number = len(clients)          
client_index = 0
headers = {}
headers['Accept'] = 'application/vnd.github.starfox-preview+json'


def get_data_pages(req_url):    
    page_number = 1
    resp_list = [] 
    
    while(True):
        print(page_number)
        number_of_tries = 10
        try:   
            r = requests.get(req_url + "&page=" + str(page_number), headers=headers)            
            if(r.ok):                            
                result = json.loads(r.text or r.content)
                resp_list += result            

                if not result:
                    break
                
                if(len(result)<100):
                    break
                    
                page_number += 1
                    
                # check for max limit
                try:
                    if int(r.headers["X-RateLimit-Remaining"]) < 10:
                        print("limit exceeded!!!!!!!!!!!!")
                        delay = float(r.headers["X-RateLimit-Reset"]) - time.mktime(time.localtime())#.total_seconds()
                        print('sleeping for '+str(delay)+' seconds...')
                        print("current time:" + str(datetime.now()))
                        time.sleep(int(delay))
                except (KeyError):
                    pass            
            else:
                j = json.loads(r.text or r.content)
                print('\n---'+str(r))
                print('\n---'+str(j['message']))
                return False
        except requests.exceptions.Timeout as e:
            print("-------timeout-------")
            print(e)
            number_of_tries-=1
            if(number_of_tries):
                time.sleep(delay_conn)
                get_data_pages(req_url)
            else:
                sys.exit(1)
        except requests.ConnectionError as e:
            print("-------connection error-------")
            print(e)
            number_of_tries-=1
            if(number_of_tries):
                time.sleep(delay_conn)
                get_data_pages(req_url)
            else:
                sys.exit(1)
    return resp_list

print("Starting time:" + str(datetime.now()))

client_id, client_secret = clients[client_index]
req = "https://api.github.com/repos/" + owner + '/' + repo + "/contributors" + "?per_page=100&client_id=" + client_id + "&client_secret=" + client_secret  
cntrb_obj = get_data_pages(req)
    
print("End time:" + str(datetime.now()))

Starting time:2020-05-11 19:35:08.342382
1
2
3
4
End time:2020-05-11 19:35:14.018309


In [11]:
len(cntrb_obj)

356

In [25]:
repo = "spring-boot"
# owner = "spring-projects"
# repo = "spring-framework"

In [27]:
d = dict()

In [28]:
for i in l:
    d[i["login"]] = i["contributions"]

In [29]:
with open("cntrb/" + repo + "_repo_cntrb.txt", 'w') as f:
    f.write(json.dumps(d , indent = 4))